In [2]:
import pandas as pd
import numpy as np
import z3
from tqdm import tqdm

In [3]:
z3.set_option("parallel.enable", True)
z3.set_option("parallel.threads.max", 1024)

In [4]:
NUM_FLOW = 10
DATA_NAME = "single0"
TOPO_NAME = "0"
task = pd.read_csv("../../dac_data/%s.csv"%DATA_NAME)[:50]
network = pd.read_csv("../../dac_data/%s_topology.csv"%TOPO_NAME)
macrotick = 100
sync_error = 0
time_out = 4 * 60 * 60

for col in ['size','period','deadline','jitter']:
    task[col] = np.ceil(task[col] / macrotick).astype(int)
for col in ['t_proc','t_prop']:
    network[col] = np.ceil(network[col] / macrotick).astype(int)



## 1. Model

In [5]:
NUM_SW = 8
NUM_ES = 8
NUM_NODE = NUM_ES + NUM_SW
NETWORK_ERROR = 0
NUM_WINDOW = 5

LCM = int(np.lcm.reduce(task['period']))
net = np.zeros(shape = (NUM_NODE, NUM_NODE))

In [6]:
NG = 10

Schedule table

In [7]:
net_var = {}

for _, row in network.iterrows():
    net_var.setdefault(row['link'], {})
    net_var[row['link']]['N'] = row['q_num']
    net_var[row['link']]['s'] = row['rate']
    net_var[row['link']]['tproc'] = row['t_proc']
    ## t is the start time of entry, if v == 0, then t is set to 0
    net_var[row['link']]['t'] = z3.IntVector('t_%s'%row['link'], NUM_WINDOW)
    ## if v == 1, then the entry is available for link
    net_var[row['link']]['v'] = z3.IntVector('v', NUM_WINDOW)
    ## if c_q == q, then the queue is used
    net_var[row['link']]['c'] = [z3.IntVector('c_%s_%d'%(row['link'], i), row['q_num']) for i in range(NUM_WINDOW
    )]
    
    net[eval(row['link'])[0], eval(row['link'])[1]] = 1

Task model

In [8]:
task_var = {}
task_attr = {}
packet_weight = {}

In [9]:
## Shortest path
def bfs_paths(graph, start, goal):
    queue = [(start, [start])]
    while queue:
        (vertex, path) = queue.pop(0)
        for _next in set(np.reshape(np.argwhere(graph[vertex] > 0),  -1)) - set(path):
            if _next == goal:
                yield path + [_next]
            else:
                queue.append((_next, path + [_next]))

In [10]:
for i, row in task.iterrows():
    task.loc[i,'route'] = str(next(bfs_paths(net, int(row['src']), int(eval(row['dst'])[0]))))
    task_var.setdefault(i, {})
    task_attr.setdefault(i, {})
    route = eval(task.loc[i, 'route'])
    
    task_attr[i]['C'] = int(row['size'])
    task_attr[i]['T'] = int(row['period']) 
    task_attr[i]['L'] = int(row['deadline'])
    task_attr[i]['t_trans'] = int(task_attr[i]['C']) * 8 
    task_attr[i]['rho'] = (int(task_attr[i]['C'] * 8) + sync_error) * (len(route) - 1) / task_attr[i]['L']
#     task_attr[i]['J'] = 1000000000 
    task_attr[i]['d'] = []
    for j in range(int(LCM / task_attr[i]['T'])):
        packet_weight[(i,j)] = task_attr[i]['L'] + j * task_attr[i]['T']
    for _i, a in enumerate(route[:-1]):
        link = str((a, route[_i + 1]))
        task_var[i].setdefault(link, {})
        task_var[i][link] = {}
        task_var[i][link]['alpha'] = []
        task_var[i][link]['beta'] = []
        task_var[i][link]['group'] = []
        for j in range(int(LCM / task_attr[i]['T'])):
            task_var[i][link]['alpha'].append(z3.Int('alpha' +  str(i) + '_' + str(link) + '_' + str(j)))
            task_var[i][link]['beta'].append(z3.Int('beta' +  str(i) + '_' + str(link) + '_' + str(j)))
            task_var[i][link]['group'].append(None)
        task_attr[i].setdefault(link, {})
        
        
        
#         task_var[i][link]['phi'] = []
#         task_var[i][link]['p'] =  z3.Int('p_' +  str(i) + '_' + str(link))
#         task_var[i][link]['T'] = np.ceil(row['period'] / net_var[str(link)]['mt'])
#         task_var[i][link]['L'] = np.ceil((row['size'] * 8 / net_var[str(link)]['s']) / net_var[str(link)]['mt'])
#         for j in range(0, int(LCM / row['period'])):
#              task_var[i][link]['phi'].append(z3.Int('phi_' +  str(i) + '_' + str(link) + '_' + str(j)))

## 2. Queue assignment

In [11]:
phat = {}
for link in net_var:
    phat.setdefault(link, [0] * 8)

for f, attr in sorted(task_attr.items(), key = lambda x:x[1]['rho'], reverse=True):
    min_h = -1
    min_value = np.inf
    for g in range(8):
        result = max([phat[link][g] for link in list(task_var[f].keys())])
        if result < min_value:
            min_h = g
            min_value = result
    
    for link in list(task_var[f].keys()):
        phat[link][min_h] += task_attr[f]['rho']
        task_attr[f][link]['q'] = min_h

## 3. Taskset decomposition

In [12]:
packet_group = {}

In [13]:
packet_weight = [x[0] for x in sorted(packet_weight.items(), key = lambda x:x[1])]

In [14]:
group_size = int(np.ceil(len(packet_weight) / NG))

In [15]:
packet_group = [packet_weight[i * group_size: (i + 1) * group_size] for i in range((len(packet_weight) + group_size - 1) // group_size )]

In [16]:
for inte, group in enumerate(packet_group):
    for i, ins in group:
        for link in task_var[i]:
            task_var[i][link]['group'][ins] = inte

## 4. SMT model

Range constraints

\begin{aligned}
&\forall n_i \in W, \quad \forall y \in[2, T], t_{i, 1}=0 \\
&\quad\left(\left(t_{i, y}=0\right) \wedge \bigwedge_{\forall z \in[y+1, T]}\left(t_{i, z}=0\right)\right) \\
&\quad \vee\left(\left(t_{i, y}>0\right) \wedge\left(1 \leq t_{i, y-1}<t_{i, y} \leq H\right)\right) \\
&\forall n_i \in W, \quad \forall y \in[1, T], \quad \forall r \in[1, Q], c_{i, y, r} \in\{0, r\} \\
&\forall n_i \in W, \quad \forall y \in[1, T], \quad \forall j \in[1, V], v_{i, y, j} \in\{0,1\} . \\
&\forall f_a \in F, \quad 1 \leq q_a \leq Q . \\
&\forall f_a \in F, \quad \forall b \in\left[1,\left|\Pi_a\right|\right), \forall k \in\left[0, \frac{H}{p_a}\right) \\
&k \times p_a+1 \leq \alpha_{a, b}^k<\beta_{a, b}^k \leq d_a+k \times p_a \\
&\beta_{a, b}^k-\alpha_{a, b}^k=\gamma_a^{+} .
\end{aligned}

In [17]:
interation = 0
s = z3.Optimize()
s.set("timeout", time_out * 1000)

In [18]:

## Constraint 1
## The first entry should be one, the paper is wrong
# for (eachSW=0;eachSW<SW;eachSW++)
# 		Z3_optimize_assert(ctx,opt,Z3_mk_eq(ctx,t[eachSW][0],one));

for link in net_var:
    s.add(net_var[link]['t'][0] == 1)

for link in net_var:
    for y in range(1, NUM_WINDOW):
        ## The paper is wrong that the frist entry should be one
        first_line = z3.And(net_var[link]['t'][y] == 0 ,z3.And([x == 0 for x in net_var[link]['t'][y+1:]]))
        second_line = z3.And(net_var[link]['t'][y] > 0,
        1 <= net_var[link]['t'][y - 1],
        net_var[link]['t'][y - 1] < net_var[link]['t'][y],
        net_var[link]['t'][y] <= LCM)
        s.add(
            z3.Or(first_line, second_line)
        )

In [19]:
## Constraint 2
for link in net_var:
    for y in range(0, NUM_WINDOW):
        for q in range(0, net_var[link]['N']):
            s.add(
                z3.Or(net_var[link]['c'][y][q] == q, net_var[link]['c'][y][q] == 0)
            )

In [20]:
## Constraint 3
for link in net_var:
    for y in range(0, NUM_WINDOW):
        s.add(
            z3.Or(net_var[link]['v'][y] == 0, net_var[link]['v'][y] == 1)
        )

In [21]:
## Constraint 4
## This one can be omitted as queue assignment has already been decided

In [22]:
## Constraint 5
for i in task_attr:
    for link in task_var[i]:
        for k in range(0, int(LCM / task_attr[i]['T'])):
            s.add(
                z3.And(
                    k * task_attr[i]['T'] + 1 <= task_var[i][link]['alpha'][k],
                    task_var[i][link]['alpha'][k] < task_var[i][link]['beta'][k],
                    task_var[i][link]['beta'][k] <= task_attr[i]['L'] + k * task_attr[i]['T'],
                    task_var[i][link]['beta'][k] - task_var[i][link]['alpha'][k] == task_attr[i]['t_trans']
                )
            )

In [23]:
## Constraint 6
for i in task_var:
    links = list(task_var[i].keys())
    for hop, link in enumerate(links[:-1]):
        for k in range(0, int(LCM / task_attr[i]['T'])):
            s.add(
                task_var[i][links[hop]]['beta'][k] + net_var[links[hop + 1]]['tproc'] < task_var[i][links[hop + 1]]['alpha'][k]
            )

In [24]:
## Constraint 7
for i in task_var:
    s_hop = list(task_var[i].keys())[0]
    e_hop = list(task_var[i].keys())[-1]
    for k in range(0, int(LCM / task_attr[i]['T'])):
        s.add(
            task_var[i][e_hop]['beta'][k] <= task_attr[i]['L'] + k * task_attr[i]['T']
        )

In [25]:
## Constraint 8
for link in net_var:
    for fa, fg in [(fa, fg) for fa in task_attr for fg in task_attr if fg > fa and link in task_var[fa] and link in task_var[fg]]:
        for k, m in [(k, m) for k in range(len(task_var[fa][link]['alpha'])) for m in range(len(task_var[fg][link]['alpha']))]:
            if task_var[fa][link]['group'][k] == interation and task_var[fg][link]['group'][m] == interation:
                ins_k = task_var[fa][link]['alpha'][k]
                ins_m = task_var[fg][link]['alpha'][m]
                s.add(
                    z3.Or(
                        task_var[fa][link]['alpha'][k] > task_var[fg][link]['beta'][m],
                        task_var[fg][link]['alpha'][m] > task_var[fa][link]['beta'][k]
                    )
                )

In [26]:
## Constraint 9
for fa, fg in [(fa, fg) for fa in task_attr for fg in task_attr if fg > fa]:
    path_i = list(task_var[fa].keys())
    path_j = list(task_var[fg].keys())
    for x_a, y_a, a_b in [(path_i[_x - 1], path_j[_y - 1], i_a_b)
                    for _x, i_a_b in enumerate(path_i) 
                    for _y, j_a_b in enumerate(path_j) 
                    if i_a_b == j_a_b and task_attr[fa][i_a_b]['q'] == task_attr[fa][j_a_b]['q']]:
        for k, m in [(k, m) for k in range(len(task_var[fa][x_a]['alpha'])) for m in range(len(task_var[fg][y_a]['alpha']))]:
            if task_var[fa][x_a]['group'][k] == interation and task_var[fg][y_a]['group'][m] == interation:
                s.add(
                    z3.Or(
                        task_var[fa][x_a]['beta'][k] + net_var[x_a]['tproc'] > task_var[fg][a_b]['alpha'][m],
                        task_var[fg][y_a]['beta'][m] + net_var[y_a]['tproc'] > task_var[fa][a_b]['alpha'][k]
                    )
                )

In [27]:
## Constraint 12 (B)
def B(x, i, k, link):
    ## Or the following things
    bb = []
    for y in range(0, NUM_WINDOW):
        big_or = z3.Or([net_var[link]['c'][y][r] == task_attr[i][link]['q'] for r in range(0, net_var[link]['N'])])
        big_and = z3.And([z3.Or(
            [net_var[link]['t'][z] == 0, 
            net_var[link]['v'][z] == 0, 
            z3.And(net_var[link]['t'][z] < net_var[link]['t'][y], 
            net_var[link]['t'][y] <= x), 
            net_var[link]['t'][y] > x]
        ) for z in range(0, NUM_WINDOW) if z != y])
        second_line = z3.And(net_var[link]['v'][y] == 1, big_or, big_and)
        inner = z3.Or(net_var[link]['t'][y] == 0, second_line)
        bb.append(inner)
    return z3.Or(bb)

In [28]:
## Constraint 11 (A)

def A(x, i, k, link, last_link):
    first_line = z3.And(
        x > task_var[i][last_link]['beta'][k],
        B(x, i, k, link)
    )
    second_line = z3.And(
        [z3.Or(h <= task_var[i][last_link]['beta'][k], z3.Not(B(h, i, k, link))) for h in range(k * task_attr[i]['T'] + 1,  x)]
    )
    return z3.And(first_line, second_line)

In [29]:
## THis one is not possible to work.

## Constriant 10
for a in tqdm(task_attr):
    ## link indicates n_i,j
    links = list(task_var[a].keys())
    for hop, link in enumerate(links[1: ]):
        ## hop = 0 means the 1-th hop, so hos is the last hop
        for k in range(0, int(LCM / task_attr[a]['T'])):
            s.add(
                z3.Or([
                    z3.And(task_var[a][link]['alpha'][k] == g, A(g, a, k, link, links[hop]))
                    for g in range(k * task_attr[a]['T'] + 1, k * task_attr[a]['T'] + task_attr[a]['L'])
                ])
            )



  0%|          | 0/50 [1:29:51<?, ?it/s]

KeyboardInterrupt

